# REopt

- API structure: https://developer.nrel.gov/api/reopt/stable/help?API_KEY=DEMO_KEY
- Models.py: https://github.com/NREL/REopt_API/blob/master/reoptjl/models.py

In [34]:
import pandas as pd
from reopt_interface import setup, create_post, run_reopt, parse_dispatch_series, calc_retail_electric_cost, plotly_stacked, calc_monthly_peaks2, calc_mae2

from scipy.optimize import minimize
import numpy as np
from scipy.optimize import minimize, LinearConstraint

# to silence warnigns
# InsecureRequestWarning: Unverified HTTPS request is being made to host 'developer.nrel.gov'. Adding certificate verification is strongly advised.
import urllib3
urllib3.disable_warnings()

pd.options.plotting.backend='plotly'

with open('api.key', 'r') as f:
    API_KEY = f.read()
    
cfg,tariff,load = setup('jpl_v4.yaml')

# Make 2019 nem prices

In [35]:
nem_prices = pd.read_csv('data/tariff/PG&E NBT EEC Values 2024 Vintage.csv',
                         index_col=0,
                         parse_dates=True,
                         comment='#')
nem_prices['True Datetime'] = pd.to_datetime(nem_prices['True Datetime'])
nem_prices['weekday'] = nem_prices.index.weekday

nem_prices.index = nem_prices['True Datetime']
nem_prices = nem_prices.sort_index()
nem_prices['true weekday'] = nem_prices.index.weekday

# uncommment to save
#nem_prices[['Price']].to_csv('data/tariff/PG&E NBT EEC Values 2024 Vintage - fake 2019.csv')

# Create Forecasts

In [36]:
df = pd.read_csv('data/load/2019_JPL_v4.csv',index_col=0,parse_dates=True)

df = df.rename(columns={'Power_kW':'True_kW'})

dfmax = df.True_kW.max()

df['Persist7d_kW'] = df['True_kW'].shift(672).values
df['Persist7d_kW'] = df['Persist7d_kW'].bfill()
print('Persist7d_kW',calc_mae2(df['True_kW'],df['Persist7d_kW'],normalize=True))

# nmae 5%
df['ScaleDn_kW'] = df['True_kW'] - df['True_kW'] * 0.2547
df['ScaleUp_kW'] = df['True_kW'] + df['True_kW'] * 0.2547
print('ScaleDn_kW',calc_mae2(df['True_kW'],df['ScaleDn_kW'],normalize=True))
print('ScaleUp_kW',calc_mae2(df['True_kW'],df['ScaleUp_kW'],normalize=True))

df['SquareDn_kW'] = df['True_kW'] - df['True_kW']**2 * 0.4106/dfmax
df['SquareUp_kW'] = df['True_kW'] + df['True_kW']**2 * 0.4106/dfmax
print('SquareDn_kW',calc_mae2(df['True_kW'],df['SquareDn_kW'],normalize=True))
print('SquareUp_kW',calc_mae2(df['True_kW'],df['SquareUp_kW'],normalize=True))

tou_levels = np.array([tariff['energyweekdayschedule'][M-1][h] for M,h in zip(df.index.month,df.index.hour)])
df['TouScaleDn_kW'] = df.True_kW.values - dfmax*0.0333 * tou_levels
df['TouScaleUp_kW'] = df.True_kW.values + dfmax*0.0333 * tou_levels
print('TouScaleDn_kW',calc_mae2(df['True_kW'],df['TouScaleDn_kW'],normalize=True))
print('TouScaleUp_kW',calc_mae2(df['True_kW'],df['TouScaleUp_kW'],normalize=True))

df['ConstantDn_kW'] = df['True_kW'] - 0.05 * df.loc[:,'True_kW'].max()
print('ConstantDn_kW',calc_mae2(df['True_kW'],df['ConstantDn_kW'],normalize=True))

df['ConstantUp_kW'] = df['True_kW'] + 0.05 * df.loc[:,'True_kW'].max()
print('ConstantUp_kW',calc_mae2(df['True_kW'],df['ConstantUp_kW'],normalize=True))

std=df['True_kW'].std()
df['Random_kW'] = df['True_kW'] + 0.0631*np.random.normal(0,1,len(df))*dfmax
print('Random_kW',calc_mae2(df['True_kW'],df['Random_kW'],normalize=True))

df.to_csv('data/load/2019_JPL_v4_fake_forecasts.csv')

forecasts = df.columns

Persist7d_kW 0.06625689334702156
ScaleDn_kW 0.0500638046742822
ScaleUp_kW 0.0500638046742822
SquareDn_kW 0.04999429108400761
SquareUp_kW 0.04999429108400761
TouScaleDn_kW 0.04995
TouScaleUp_kW 0.04995
ConstantDn_kW 0.05
ConstantUp_kW 0.05
Random_kW 0.050429792902677094


### Choose coefficients for fake forecasts

In [37]:
d = df['True_kW']/dfmax

s = np.random.normal(0, 1, 35040)
def forecast(x):
    #return d + d * x[0]
    #return d + d**2 * x[0]
    #return d + x[0]
    return d + x[0]*tou_levels
    #return d+s*x[0]

def obj(x):
    return (np.mean(np.abs(d-forecast(x))) - 0.05)**2

linear_constraint = LinearConstraint([[1]], [0], [100])

r = minimize(obj,np.array([1.1]),options={'disp':True},constraints=[linear_constraint])

print('Optimal values of x:',r.x)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0109344276801338e-14
            Iterations: 3
            Function evaluations: 6
            Gradient evaluations: 3
Optimal values of x: [0.03333327]


# Perfect

In [38]:
post = create_post(cfg)
api_response = run_reopt(post,api_key=API_KEY)
dispatch_perfect = parse_dispatch_series(api_response)
costs_perfect = calc_retail_electric_cost(dispatch_perfect,tariff,cfg)

costs_perfect

Trying API request: 1
Success!


{'energy': 20398.5, 'demand': 10612.0, 'sellback': 19871.3, 'total': 11139.2}

# Solar Only

In [39]:
post = create_post(cfg,batt_kw=0,batt_h=0)

api_response = run_reopt(post,api_key=API_KEY)
dispatch_solar = parse_dispatch_series(api_response)
costs_solar = calc_retail_electric_cost(dispatch_solar,tariff,cfg)

costs_solar

Trying API request: 1
Success!


{'energy': 43511.8, 'demand': 66955.0, 'sellback': 31989.7, 'total': 78477.1}

# Forecast

In [40]:
forecast = 'Persist7d_kw'
post = create_post(cfg,load_col=forecast)

api_response = run_reopt(post,api_key=API_KEY)
dispatch_forecast = parse_dispatch_series(api_response)
costs_forecast = calc_retail_electric_cost(dispatch_forecast,tariff,cfg)
costs_forecast

Trying API request: 1
Success!


{'energy': 21048.800000000003,
 'demand': 11416.0,
 'sellback': 21574.300000000003,
 'total': 10890.5}

# Actual

In [41]:
dispatch = dispatch_forecast.copy()

dispatch['load_forecast'] = dispatch['load']
dispatch['load'] = load.True_kW
dispatch['grid'] = dispatch.load - dispatch.pv - dispatch.batt

costs_actual = calc_retail_electric_cost(dispatch,tariff,cfg)
costs_actual

{'energy': 33436.100000000006,
 'demand': 54841.0,
 'sellback': 26489.7,
 'total': 61787.40000000001}

# Plot

## Actual

In [42]:
month = 5
peak_hours = list(range(9,21))

In [58]:
monthly_peaks = calc_monthly_peaks2(dispatch.grid,peak_hours)
if monthly_peaks is not None:
    peak = monthly_peaks.iloc[month-1]['grid kw']
    t_peak = monthly_peaks.iloc[month-1]['grid t']
else:
    peak = None
    t_peak = None   

begin = t_peak - pd.Timedelta('1h')*36
end = t_peak + pd.Timedelta('1h')*36

f = plotly_stacked(dispatch.loc[f'2019-{month}'],#[begin:end],
                    solar='pv',
                    load='load',
                    load_forecast='load_forecast',
                    batt='batt',
                    utility='grid',
                    soc='soc',
                    theme='plotly',
                    title=f'{forecast} Forecast<br>Monthly Peak {peak} kW at {t_peak}')

## Perfect

In [44]:
load_at_peak = dispatch_perfect.loc[t_peak,'grid']

f = plotly_stacked(dispatch_perfect.loc[f'2019-{month}'],#[begin:end],
                    solar='pv',
                    load='load',
                    load_forecast='load_forecast',
                    batt='batt',
                    utility='grid',
                    soc='soc',
                    theme='plotly',
                    title=f'Load {load_at_peak:.1f} kW at {t_peak}')

# Multiple Forecasts

In [45]:
forecasts = ['SolarOnly'] + forecasts.to_list()
forecasts

['Perfect',
 'SolarOnly',
 'True_kW',
 'Persist7d_kW',
 'ScaleDn_kW',
 'ScaleUp_kW',
 'SquareDn_kW',
 'SquareUp_kW',
 'TouScaleDn_kW',
 'TouScaleUp_kW',
 'ConstantDn_kW',
 'ConstantUp_kW',
 'Random_kW']

In [46]:
# Ce cost of energy
# Cd cost of demand
# Re revenue of energy
# Ct total cost
# p_x is performance based on Ce, Cd, Re, or Ct
# pk_x peak of month x
# tpk_x time of peak of month x
r = pd.DataFrame(columns=[  'Ce','Cd','Re','Ct',
                            'p_Ce','p_Cd','p_Re','p_Ct',
                            'pk_1','pk_2','pk_3','pk_4','pk_5','pk_6','pk_7','pk_8','pk_9','pk_10','pk_11','pk_12',
                            'tpk_1','tpk_2','tpk_3','tpk_4','tpk_5','tpk_6','tpk_7','tpk_8','tpk_9','tpk_10','tpk_11','tpk_12',
                            'dispatch'],
                       index=forecasts)

Insert perfect results

In [47]:
peaks = calc_monthly_peaks2(dispatch_perfect.grid,peak_hours)['grid kw'].values
t_peaks = calc_monthly_peaks2(dispatch_perfect.grid,peak_hours)['grid t'].values

r.loc['Perfect'] = {'Ce':costs_perfect.energy,'Cd':costs_perfect.demand,'Re':costs_perfect.sellback,'Ct':costs_perfect.total,
                    'p_Ce':0,'p_Cd':0,'p_Re':0,'p_Ct':0,
                    'pk_1':peaks[0],'pk_2':peaks[1],'pk_3':peaks[2],'pk_4':peaks[3],'pk_5':peaks[4],'pk_6':peaks[5],
                    'pk_7':peaks[6],'pk_8':peaks[7],'pk_9':peaks[8],'pk_10':peaks[9],'pk_11':peaks[10],'pk_12':peaks[11],
                    'tpk_1':t_peaks[0],'tpk_2':t_peaks[1],'tpk_3':t_peaks[2],'tpk_4':t_peaks[3],'tpk_5':t_peaks[4],'tpk_6':t_peaks[5],
                    'tpk_7':t_peaks[6],'tpk_8':t_peaks[7],'tpk_9':t_peaks[8],'tpk_10':t_peaks[9],'tpk_11':t_peaks[10],'tpk_12':t_peaks[11],
                    'dispatch':dispatch_perfect}

Insert solar-only results

In [48]:
peaks =   calc_monthly_peaks2(dispatch_solar.grid,peak_hours)['grid kw'].values
t_peaks = calc_monthly_peaks2(dispatch_solar.grid,peak_hours)['grid t'].values

r.loc['SolarOnly'] = {'Ce':costs_solar.energy,'Cd':costs_solar.demand,'Re':costs_solar.sellback,'Ct':costs_solar.total,
                    'p_Ce':0,'p_Cd':0,'p_Re':0,'p_Ct':0,
                    'pk_1':peaks[0],'pk_2':peaks[1],'pk_3':peaks[2],'pk_4':peaks[3],'pk_5':peaks[4],'pk_6':peaks[5],
                    'pk_7':peaks[6],'pk_8':peaks[7],'pk_9':peaks[8],'pk_10':peaks[9],'pk_11':peaks[10],'pk_12':peaks[11],
                    'tpk_1':t_peaks[0],'tpk_2':t_peaks[1],'tpk_3':t_peaks[2],'tpk_4':t_peaks[3],'tpk_5':t_peaks[4],'tpk_6':t_peaks[5],
                    'tpk_7':t_peaks[6],'tpk_8':t_peaks[7],'tpk_9':t_peaks[8],'tpk_10':t_peaks[9],'tpk_11':t_peaks[10],'tpk_12':t_peaks[11],
                    'dispatch':dispatch_solar}

In [49]:
for fcast in forecasts[2:]:    
    print('///// ',fcast,' /////')
    post = create_post(cfg,load_col=fcast)
    api_response = run_reopt(post,api_key=API_KEY)
    d = parse_dispatch_series(api_response)

    # make the dispatch actual
    d['load_forecast'] = d['load']
    d['load'] = load.True_kW
    d['grid'] = d.load - d.pv - d.batt

    c = calc_retail_electric_cost(d,tariff,cfg)
    
    pks =   calc_monthly_peaks2(d.grid,peak_hours)['grid kw'].values
    tpks = calc_monthly_peaks2(d.grid,peak_hours)['grid t'].values
    
    r.loc[fcast] = {'Ce':c.energy,'Cd':c.demand,'Re':c.sellback,'Ct':c.total,
                    'p_Ce':round(1 - (c.energy-costs_perfect.energy)/(costs_solar.energy-costs_perfect.energy),3),
                    'p_Cd':round(1 - (c.demand-costs_perfect.demand)/(costs_solar.demand-costs_perfect.demand),3),
                    'p_Re':round(1 - (c.sellback-costs_perfect.sellback)/(costs_solar.sellback-costs_perfect.sellback),3),
                    'p_Ct':round(1 - (c.total-costs_perfect.total)/(costs_solar.total-costs_perfect.total),3),
                    'pk_1':pks[0],'pk_2':pks[1],'pk_3':pks[2],'pk_4':pks[3],'pk_5':pks[4],'pk_6':pks[5],
                    'pk_7':pks[6],'pk_8':pks[7],'pk_9':pks[8],'pk_10':pks[9],'pk_11':pks[10],'pk_12':pks[11],
                    'tpk_1':tpks[0],'tpk_2':tpks[1],'tpk_3':tpks[2],'tpk_4':tpks[3],'tpk_5':tpks[4],'tpk_6':tpks[5],
                    'tpk_7':tpks[6],'tpk_8':tpks[7],'tpk_9':tpks[8],'tpk_10':tpks[9],'tpk_11':tpks[10],'tpk_12':tpks[11],
                    'dispatch':d}
    

/////  True_kW  /////
Trying API request: 1
Success!
/////  Persist7d_kW  /////
Trying API request: 1
Success!
/////  ScaleDn_kW  /////
Trying API request: 1
Success!
/////  ScaleUp_kW  /////
Trying API request: 1
Success!
/////  SquareDn_kW  /////
Trying API request: 1
Success!
/////  SquareUp_kW  /////
Trying API request: 1
Success!
/////  TouScaleDn_kW  /////
Trying API request: 1
Trying API request: 1
Success!
/////  TouScaleUp_kW  /////
Trying API request: 1
Success!
/////  ConstantDn_kW  /////
Trying API request: 1
Success!
/////  ConstantUp_kW  /////
Trying API request: 1
Success!
/////  Random_kW  /////
Trying API request: 1
Trying API request: 1
Success!


In [55]:
r

,Ce,Cd,Re,Ct,p_Ce,p_Cd,p_Re,p_Ct,pk_1,pk_2,...,tpk_4,tpk_5,tpk_6,tpk_7,tpk_8,tpk_9,tpk_10,tpk_11,tpk_12,dispatch
SolarOnly,43511.8,66955.0,31989.7,78477.1,0,0,0,0,630.5,675.3,...,2019-04-16T09:00:00.000000000,2019-05-13T09:00:00.000000000,2019-06-20T09:00:00.000000000,2019-07-25T09:00:00.000000000,2019-08-06T09:00:00.000000000,2019-09-26T10:00:00.000000000,2019-10-07T09:00:00.000000000,2019-11-28T09:00:00.000000000,2019-12-26T09:00:00.000000000,load pv grid 2...
True_kW,20398.5,10612.0,19871.3,11139.2,1.0,1.0,1.0,1.0,104.5,118.5,...,2019-04-16T09:00:00.000000000,2019-05-08T18:00:00.000000000,2019-06-02T18:00:00.000000000,2019-07-25T20:00:00.000000000,2019-08-04T20:00:00.000000000,2019-09-26T16:00:00.000000000,2019-10-27T15:00:00.000000000,2019-11-28T17:00:00.000000000,2019-12-23T16:00:00.000000000,load pv batt soc ...
Persist7d_kW,33436.1,54841.0,26489.7,61787.4,0.436,0.215,0.454,0.248,681.6,436.0,...,2019-04-19T09:00:00.000000000,2019-05-23T11:00:00.000000000,2019-06-14T09:00:00.000000000,2019-07-26T09:00:00.000000000,2019-08-23T09:00:00.000000000,2019-09-20T09:00:00.000000000,2019-10-18T09:00:00.000000000,2019-11-15T09:00:00.000000000,2019-12-13T09:00:00.000000000,load pv batt ...
ScaleDn_kW,34373.9,24211.0,24846.5,33738.4,0.395,0.759,0.589,0.664,182.5,193.7,...,2019-04-02T09:00:00.000000000,2019-05-08T09:00:00.000000000,2019-06-24T09:00:00.000000000,2019-07-17T09:00:00.000000000,2019-08-28T09:00:00.000000000,2019-09-26T09:00:00.000000000,2019-10-02T09:00:00.000000000,2019-11-20T09:00:00.000000000,2019-12-03T09:00:00.000000000,load pv batt soc...
ScaleUp_kW,29658.3,15952.0,27036.1,18574.2,0.599,0.905,0.409,0.89,160.0,171.8,...,2019-04-29T14:00:00.000000000,2019-05-08T18:00:00.000000000,2019-06-10T18:00:00.000000000,2019-07-25T17:00:00.000000000,2019-08-04T20:00:00.000000000,2019-09-26T18:00:00.000000000,2019-10-27T15:00:00.000000000,2019-11-28T17:00:00.000000000,2019-12-23T17:00:00.000000000,load pv batt ...
SquareDn_kW,34432.5,34316.0,25223.8,43524.7,0.393,0.579,0.558,0.519,251.5,275.4,...,2019-04-02T09:00:00.000000000,2019-05-08T09:00:00.000000000,2019-06-24T09:00:00.000000000,2019-07-17T09:00:00.000000000,2019-08-28T09:00:00.000000000,2019-09-26T09:00:00.000000000,2019-10-02T09:00:00.000000000,2019-11-20T09:00:00.000000000,2019-12-03T09:00:00.000000000,load pv batt soc ...
SquareUp_kW,29123.4,19300.0,26624.4,21799.0,0.623,0.846,0.443,0.842,184.0,247.3,...,2019-04-16T15:00:00.000000000,2019-05-08T17:00:00.000000000,2019-06-10T18:00:00.000000000,2019-07-25T17:00:00.000000000,2019-08-26T20:00:00.000000000,2019-09-26T17:00:00.000000000,2019-10-21T18:00:00.000000000,2019-11-28T17:00:00.000000000,2019-12-23T17:00:00.000000000,load pv batt ...
TouScaleDn_kW,28767.9,12151.0,22314.5,18604.4,0.638,0.973,0.798,0.889,104.5,118.5,...,2019-04-16T09:00:00.000000000,2019-05-08T18:00:00.000000000,2019-06-26T19:00:00.000000000,2019-07-04T19:00:00.000000000,2019-08-01T18:00:00.000000000,2019-09-26T16:00:00.000000000,2019-10-27T15:00:00.000000000,2019-11-28T17:00:00.000000000,2019-12-23T16:00:00.000000000,load pv batt soc ...
TouScaleUp_kW,27043.2,10612.0,24758.0,12897.2,0.713,1.0,0.597,0.974,104.5,118.5,...,2019-04-16T09:00:00.000000000,2019-05-08T18:00:00.000000000,2019-06-02T18:00:00.000000000,2019-07-14T19:00:00.000000000,2019-08-04T20:00:00.000000000,2019-09-26T16:00:00.000000000,2019-10-27T15:00:00.000000000,2019-11-28T17:00:00.000000000,2019-12-23T16:00:00.000000000,load pv batt ...
ConstantDn_kW,27142.2,11770.0,22021.5,16890.7,0.708,0.979,0.823,0.915,104.5,118.5,...,2019-04-16T09:00:00.000000000,2019-05-08T18:00:00.000000000,2019-06-28T19:00:00.000000000,2019-07-25T20:00:00.000000000,2019-08-09T18:00:00.000000000,2019-09-26T16:00:00.000000000,2019-10-27T15:00:00.000000000,2019-11-28T17:00:00.000000000,2019-12-23T16:00:00.000000000,load pv batt soc ...


In [51]:
results[[f'peak_m{i}' for i in range(1,13)]].T

,Perfect,SolarOnly,True_kW,Persist7d_kW,ScaleDn_kW,ScaleUp_kW,SquareDn_kW,SquareUp_kW,TouScaleDn_kW,TouScaleUp_kW,ConstantDn_kW,ConstantUp_kW,Random_kW
peak_m1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
peak_m2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
peak_m3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
peak_m4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
peak_m5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
peak_m6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
peak_m7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
peak_m8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
peak_m9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
peak_m10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
results[[f'peak_m{i}' for i in range(1,13)]].T.plot()